In [1]:
from langchain.llms import Ollama; 
from langchain.embeddings import OllamaEmbeddings
from langchain.docstore.base import Document
from langchain.document_loaders import WebBaseLoader
from langchain.vectorstores import Chroma
from langchain.document_loaders import WebBaseLoader



In [2]:
embeddings = OllamaEmbeddings()
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

        


In [3]:
# Split into chunks 
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
all_splits = text_splitter.split_documents(data)

In [4]:
from langchain.vectorstores import ChromallEmbeddings
from langchain.embeddings import OllamaEmbeddings # We can also try Ollama embeddings
vectorstore = Chroma.from_documents(documents=all_splits,
                                    embedding=OllamaEmbeddings())

In [5]:
# Retrieve
question = "How can Task Decomposition be done?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [6]:
docs[0]

Document(page_content='Resources:\n1. Internet access for searches and information gathering.\n2. Long Term memory management.\n3. GPT-3.5 powered Agents for delegation of simple tasks.\n4. File output.\n\nPerformance Evaluation:\n1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.\n2. Constructively self-criticize your big-picture behavior constantly.\n3. Reflect on past decisions and strategies to refine your approach.\n4. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.', metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM

In [7]:


QA_CHAIN_PROMPT = """
[INST]<<SYS>> You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.<</SYS>> 
Question: {question} 
Context: {context} 
Answer: [/INST]
"""

In [ ]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
llm = Ollama(model="llama2",
             verbose=True)

In [ ]:
prompt = "Write a joke about a {subject}"
pt = PromptTemplate(template=prompt, input_variables=["subject"])
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt = pt)
result = chain("potato")

eval = """ why this joke below funny? Explain the humour.
   Joke: {joke}
   """
print (result)
eval_prompt = PromptTemplate(template= eval, input_variables=["joke"])
eval_chain = LLMChain(llm=llm, prompt=eval_prompt)
eval_chain(result['text'])





In [18]:

from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
llm = Ollama(model="llama2",
             verbose=True)
prompt = PromptTemplate(template= QA_CHAIN_PROMPT,input_variables=["question", "context"])
qa_chain = RetrievalQA.from_chain_type(
    llm,retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt":prompt}
) 

In [19]:
question = "What are the various approaches to Task Decomposition for AI Agents?"
result = qa_chain({"query": question})

In [20]:
result

{'query': 'What are the various approaches to Task Decomposition for AI Agents?',
 'result': 'Task decomposition for AI agents involves breaking down complex tasks into smaller, more manageable sub-tasks that can be executed by the agent. There are various approaches to task decomposition, including:\n\n1. Divide and Conquer: This approach involves dividing a task into smaller sub-tasks, each of which can be executed independently. The results from each sub-task are then combined to achieve the overall goal.\n2. Hierarchical Task Networks (HTN): This approach involves breaking down a task into a hierarchy of sub-tasks, with each level of the hierarchy representing a different level of detail or abstraction. The agent executes the tasks at each level in a hierarchical manner.\n3. Workflow Nets: This approach involves modeling a task as a network of interconnected sub-tasks, with each sub-task representing a specific action or operation. The agent executes the tasks in a serial or parall